# Imports

In [73]:
from env import user, password, host
import prepare

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Acquire

In [2]:
# Create a SQL query to get the data from SQL
query = '''
SELECT *
FROM customers
JOIN contract_types USING(`contract_type_id`)
JOIN internet_service_types USING(`internet_service_type_id`)
JOIN payment_types USING(`payment_type_id`)
'''

In [3]:
# Get the url for the SQL database, letting us query the telco_churn database
telco_url = f'mysql+pymysql://{user}:{password}@{host}/telco_churn'

In [4]:
telco = pd.read_sql(query, telco_url)

In [5]:
telco.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
1,4,1,1,0013-MHZWF,Female,0,No,Yes,9,Yes,No,No,No,No,Yes,Yes,Yes,Yes,69.4,571.45,No,Month-to-month,DSL,Credit card (automatic)
2,1,1,1,0015-UOCOJ,Female,1,No,No,7,Yes,No,Yes,No,No,No,No,No,Yes,48.2,340.35,No,Month-to-month,DSL,Electronic check
3,1,1,1,0023-HGHWL,Male,1,No,No,1,No,No phone service,No,No,No,No,No,No,Yes,25.1,25.1,Yes,Month-to-month,DSL,Electronic check
4,3,1,1,0032-PGELS,Female,0,Yes,Yes,1,No,No phone service,Yes,No,No,No,No,No,No,30.5,30.5,Yes,Month-to-month,DSL,Bank transfer (automatic)


## Create a Function and Test It

In [6]:
def get_telco_data():
    # Create a SQL query to get the data from SQL
    query = '''
            SELECT *
            FROM customers
            JOIN contract_types USING(`contract_type_id`)
            JOIN internet_service_types USING(`internet_service_type_id`)
            JOIN payment_types USING(`payment_type_id`)
            '''
    
    # Get the url for the SQL database, letting us query the telco_churn database
    telco_url = f'mysql+pymysql://{user}:{password}@{host}/telco_churn'
        
    return pd.read_sql(query, telco_url)

In [7]:
df = get_telco_data()
df.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
1,4,1,1,0013-MHZWF,Female,0,No,Yes,9,Yes,No,No,No,No,Yes,Yes,Yes,Yes,69.4,571.45,No,Month-to-month,DSL,Credit card (automatic)
2,1,1,1,0015-UOCOJ,Female,1,No,No,7,Yes,No,Yes,No,No,No,No,No,Yes,48.2,340.35,No,Month-to-month,DSL,Electronic check
3,1,1,1,0023-HGHWL,Male,1,No,No,1,No,No phone service,No,No,No,No,No,No,Yes,25.1,25.1,Yes,Month-to-month,DSL,Electronic check
4,3,1,1,0032-PGELS,Female,0,Yes,Yes,1,No,No phone service,Yes,No,No,No,No,No,No,30.5,30.5,Yes,Month-to-month,DSL,Bank transfer (automatic)


# Prepare

In [8]:
train, test, validate = prepare.prep_telco(telco, train_size=.8, seed=123)

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4507 entries, 1249 to 6958
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customer_id            4507 non-null   object 
 1   gender                 4507 non-null   object 
 2   senior_citizen         4507 non-null   int64  
 3   partner                4507 non-null   object 
 4   dependents             4507 non-null   object 
 5   tenure                 4507 non-null   int64  
 6   phone_service          4507 non-null   object 
 7   multiple_lines         4507 non-null   object 
 8   online_security        4507 non-null   object 
 9   online_backup          4507 non-null   object 
 10  device_protection      4507 non-null   object 
 11  tech_support           4507 non-null   object 
 12  streaming_tv           4507 non-null   object 
 13  streaming_movies       4507 non-null   object 
 14  paperless_billing      4507 non-null   object 
 15  m

## Split the Data

In [10]:
train, test = train_test_split(telco, train_size=.8, random_state=123)

In [11]:
train, validate = train_test_split(train, train_size=.8, random_state=123)

In [12]:
print(train.shape, test.shape, validate.shape)

(4507, 24) (1409, 24) (1127, 24)


## Create a Function and Test It

In [13]:
def split_data(df, train_size, seed):
    # Create the train and test sets
    train, test = train_test_split(df, train_size=train_size, random_state=seed)
    # Create the validate set by splitting from train set
    train, validate = train_test_split(train, train_size=train_size, random_state=seed)
    
    return train, test, validate

In [14]:
train, test, validate = split_data(telco, train_size=.8, seed=123)

In [15]:
print(train.shape, test.shape, validate.shape)

(4507, 24) (1409, 24) (1127, 24)


## Clean the Data

In [16]:
telco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   payment_type_id           7043 non-null   int64  
 1   internet_service_type_id  7043 non-null   int64  
 2   contract_type_id          7043 non-null   int64  
 3   customer_id               7043 non-null   object 
 4   gender                    7043 non-null   object 
 5   senior_citizen            7043 non-null   int64  
 6   partner                   7043 non-null   object 
 7   dependents                7043 non-null   object 
 8   tenure                    7043 non-null   int64  
 9   phone_service             7043 non-null   object 
 10  multiple_lines            7043 non-null   object 
 11  online_security           7043 non-null   object 
 12  online_backup             7043 non-null   object 
 13  device_protection         7043 non-null   object 
 14  tech_sup

- Surprisingly, it looks like we don't have any nulls within the data
- We can patch out the type id's, since they were only useful for merging
    * We could've also done this in SQL, but I prefer getting to python as soon as possible
- Fields to look at:
    * gender: Currently an object, likely needs to be encoded
    * senior_citizen: It's an int type, does that mean it's encoded already?
    * partner: I have no clue what this field is for
    * dependents: Currently an object, either it's a bool or should be an int identifying how many dependents
    * phone_service - paperless_billing: Needs to be encoded
    * total_charges: Definitely should not be an object, likely needs to be a float
    * churn: Probably needs to be encoded


## Drop Columns

In [17]:
# Dropping all of the columns used just to merge the data together in the SQL query. These columns will provide nothing good for our models or exploration
telco.drop(columns=['payment_type_id',
                    'internet_service_type_id',
                    'contract_type_id'], inplace=True)

In [18]:
telco.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,0003-MKNFE,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
1,0013-MHZWF,Female,0,No,Yes,9,Yes,No,No,No,No,Yes,Yes,Yes,Yes,69.4,571.45,No,Month-to-month,DSL,Credit card (automatic)
2,0015-UOCOJ,Female,1,No,No,7,Yes,No,Yes,No,No,No,No,No,Yes,48.2,340.35,No,Month-to-month,DSL,Electronic check
3,0023-HGHWL,Male,1,No,No,1,No,No phone service,No,No,No,No,No,No,Yes,25.1,25.1,Yes,Month-to-month,DSL,Electronic check
4,0032-PGELS,Female,0,Yes,Yes,1,No,No phone service,Yes,No,No,No,No,No,No,30.5,30.5,Yes,Month-to-month,DSL,Bank transfer (automatic)


A little bit more manageable

## Fixing Datatypes

First, looking to patch up any features which are leading to type errors (looking at you total_charges)

In [19]:
# Check to see if there's a space in total_charges
telco[telco.total_charges.str.contains(' ')]

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
1878,1371-DWPAZ,Female,0,Yes,Yes,0,No,No phone service,Yes,Yes,Yes,Yes,Yes,No,No,56.05,,No,Two year,DSL,Credit card (automatic)
1949,2775-SEFEE,Male,0,No,Yes,0,Yes,Yes,Yes,Yes,No,Yes,No,No,Yes,61.90,,No,Two year,DSL,Bank transfer (automatic)
2029,4075-WKNIU,Female,0,Yes,Yes,0,Yes,Yes,No,Yes,Yes,Yes,Yes,No,No,73.35,,No,Two year,DSL,Mailed check
2048,4472-LVYGI,Female,0,Yes,Yes,0,No,No phone service,Yes,No,Yes,Yes,Yes,No,Yes,52.55,,No,Two year,DSL,Bank transfer (automatic)
2132,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,Yes,Yes,Yes,No,Yes,Yes,No,80.85,,No,Two year,DSL,Mailed check
6143,2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Yes,19.70,,No,One year,None,Mailed check
6569,2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.00,,No,Two year,None,Mailed check
6605,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.25,,No,Two year,None,Mailed check
6615,3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,25.35,,No,Two year,None,Mailed check
6686,4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,25.75,,No,Two year,None,Mailed check


Looks like we found the culprit: there's customers who haven't paid their first check yet, so total charges doesn't reflect it.
This leads to the first real choice for this project, do we impute or do we drop?
It would be best not to use these individuals for training, since none have churned and likely doesn't provide real value for train. But!
We could take this subset of customers and use them as a perfect example for a sub-test to test our predictions on.

Find the total_charges == ' ', then replace them with zero

In [20]:
telco.total_charges = telco.total_charges.str.replace(' ', '0')

In [21]:
telco.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,0003-MKNFE,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
1,0013-MHZWF,Female,0,No,Yes,9,Yes,No,No,No,No,Yes,Yes,Yes,Yes,69.4,571.45,No,Month-to-month,DSL,Credit card (automatic)
2,0015-UOCOJ,Female,1,No,No,7,Yes,No,Yes,No,No,No,No,No,Yes,48.2,340.35,No,Month-to-month,DSL,Electronic check
3,0023-HGHWL,Male,1,No,No,1,No,No phone service,No,No,No,No,No,No,Yes,25.1,25.1,Yes,Month-to-month,DSL,Electronic check
4,0032-PGELS,Female,0,Yes,Yes,1,No,No phone service,Yes,No,No,No,No,No,No,30.5,30.5,Yes,Month-to-month,DSL,Bank transfer (automatic)


## Encode

In [22]:
def encode_contract_types(train, test, validate):
    '''Takes in train, test and validate dataframes
    Returns each df with a new coloumn for encoded contract types 
    as well as the encoder used'''
    encoder = LabelEncoder()
    encoder.fit(train.contract_type)
    train["contract_type_encoded"] = encoder.transform(train.contract_type)
    test["contract_type_encoded"] = encoder.transform(test.contract_type)
    validate["contract_type_encoded"] = encoder.transform(validate.contract_type)
    return encoder, train, test, validate

In [23]:
def encode_internet_service_types(train, test, validate):
    '''Takes in train, test and validate dataframes
    Returns each df with a new coloumn for encoded contract types 
    as well as the encoder used'''
    encoder = LabelEncoder()
    encoder.fit(train.internet_service_type)
    train["internet_service_type"] = encoder.transform(train.internet_service_type)
    test["internet_service_type"] = encoder.transform(test.internet_service_type)
    validate["internet_service_type"] = encoder.transform(validate.internet_service_type)
    return encoder, train, test, validate

In [24]:
def encode_payment_type(train, test, validate):
    '''Takes in train, test and validate dataframes
    Returns each df with a new coloumn for encoded contract types 
    as well as the encoder used'''
    encoder = LabelEncoder()
    encoder.fit(train.payment_type)
    train["payment_type_encoded"] = encoder.transform(train.payment_type)
    test["payment_type_encoded"] = encoder.transform(test.payment_type)
    validate["payment_type_encoded"] = encoder.transform(validate.payment_type)
    return encoder, train, test, validate

In [25]:
def encode_churn(train, test, validate):
    '''Takes in train, test and validate dataframes
    Returns each df with a new coloumn for encoded contract types 
    as well as the encoder used'''
    encoder = LabelEncoder()
    encoder.fit(train.churn)
    train["churn_encoded"] = encoder.transform(train.churn)
    test["churn_encoded"] = encoder.transform(test.churn)
    validate["churn_encoded"] = encoder.transform(validate.churn)
    return encoder, train, test, validate

In [26]:
def encode_phone_service(train, test, validate):
    '''Takes in train, test and validate dataframes
    Returns each df with a new coloumn for encoded contract types 
    as well as the encoder used'''
    encoder = LabelEncoder()
    encoder.fit(train.phone_service)
    train["phone_service_encoded"] = encoder.transform(train.phone_service)
    test["phone_service_encoded"] = encoder.transform(test.phone_service)
    validate["phone_service_encoded"] = encoder.transform(validate.phone_service)
    return encoder, train, test, validate

In [27]:
def encoded_df(train, test, validate):
    encoder_1, train, test, validate = encode_contract_types(train, test, validate)
    encoder_2, train, test, validate = encode_internet_service_types(train, test, validate)
    encoder_3, train, test, validate = encode_churn(train, test, validate)
    encoder_4, train, test, validate = encode_payment_type(train, test, validate)
    encoder_5, train, test, validate = encode_phone_service(train, test, validate)
    return encoder_1, encoder_2, encoder_3, encoder_4, encoder_5, train, test, validate

In [28]:
encoder_1, encoder_2, encoder_3, encoder_4, encoder_5, train, test, validate = encoded_df(train, test, validate)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Modeling

In [50]:
X_train = train[['tenure', 'contract_type_encoded', 'monthly_charges', 'payment_type_encoded']]
y_train = train.churn

X_validate = validate[['tenure', 'contract_type_encoded', 'monthly_charges', 'payment_type_encoded']]
y_validate = validate.churn

In [51]:
predictions = pd.DataFrame({'actual' : y_validate})

## Baseline

In [52]:
predictions['baseline'] = 'Yes'

In [53]:
predictions.head()

,actual,baseline
5854,No,Yes
797,No,Yes
3188,No,Yes
2254,No,Yes
5732,No,Yes


## Logistic Regregression

In [54]:
log_reg_model = LogisticRegression(random_state=123).fit(X_train, y_train)

In [55]:
predictions['logistic Regression: churn ~ tenure, contract type, monthly charges'] = log_reg_model.predict(X_validate)

In [56]:
predictions.head()

,actual,baseline,"logistic Regression: churn ~ tenure, contract type, monthly charges"
5854,No,Yes,No
797,No,Yes,No
3188,No,Yes,No
2254,No,Yes,No
5732,No,Yes,No


In [57]:
log_reg_model.score(X_validate, y_validate)

0.8012422360248447

## Decision Tree 

In [58]:
tree_model = DecisionTreeClassifier(max_depth=3, random_state=123).fit(X_train, y_train)

In [59]:
predictions['Decision Tree: churn ~ tenure, contract type, monthly charges'] = tree_model.predict(X_validate)

In [60]:
predictions.head()

,actual,baseline,"logistic Regression: churn ~ tenure, contract type, monthly charges","Decision Tree: churn ~ tenure, contract type, monthly charges"
5854,No,Yes,No,No
797,No,Yes,No,No
3188,No,Yes,No,No
2254,No,Yes,No,No
5732,No,Yes,No,No


In [61]:
tree_model.score(X_validate, y_validate)

0.7985803016858918

## Random Forest

In [62]:
forest_model = RandomForestClassifier(n_estimators=10, random_state=123).fit(X_train, y_train)

In [63]:
predictions['Random Forest: churn ~ tenure, contract type, monthly charges'] = forest_model.predict(X_validate)

In [64]:
forest_model.score(X_validate, y_validate)

0.7897071872227152

## K Nearest Neighbors 

In [65]:
knn = KNeighborsClassifier(n_neighbors=5).fit(X_train, y_train)

In [66]:
predictions['KNN: churn ~ tenure, contract type, monthly charges'] = knn.predict(X_validate)

In [67]:
knn.score(X_train, y_train)

0.8335921899267805

In [68]:
predictions.head()

,actual,baseline,"logistic Regression: churn ~ tenure, contract type, monthly charges","Decision Tree: churn ~ tenure, contract type, monthly charges","Random Forest: churn ~ tenure, contract type, monthly charges","KNN: churn ~ tenure, contract type, monthly charges"
5854,No,Yes,No,No,No,No
797,No,Yes,No,No,No,No
3188,No,Yes,No,No,No,No
2254,No,Yes,No,No,No,No
5732,No,Yes,No,No,No,No


In [69]:
predictions[['actual', 'KNN: churn ~ tenure, contract type, monthly charges']]

,actual,"KNN: churn ~ tenure, contract type, monthly charges"
5854,No,No
797,No,No
3188,No,No
2254,No,No
5732,No,No
...,...,...
4811,No,No
6879,No,No
5366,Yes,No
1301,No,No


In [70]:
predictions.to_csv('model_output.csv')

# Evaluate

## Confusion Matrix

In [75]:
# Logistic Regression
confusion_matrix(predictions.actual, log_reg_model.predict(X_validate))

array([[767,  78],
       [146, 136]])

In [77]:
# Decision Tree
confusion_matrix(predictions.actual, tree_model.predict(X_validate))

array([[788,  57],
       [170, 112]])

In [78]:
# Random Forest
confusion_matrix(predictions.actual, forest_model.predict(X_validate))

array([[752,  93],
       [144, 138]])

In [79]:
# KNN
confusion_matrix(predictions.actual, knn.predict(X_validate))

array([[746,  99],
       [143, 139]])